# Basic Simulator For CatBot

Simulate CatBot with no gravity.  Allows for manual control of joints.

Based on: https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-ad3d32e7918847799644d6ffae34866f

In [1]:
import numpy as np

from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ModelVisualizer

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [4]:
model_path = './catBot.urdf'

## Show Model

In [5]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(model_path)

visualizer.Run(loop_once=False)

Click 'Stop Running' or press Esc to quit


<RunResult.STOPPED: 2>

In [10]:
def create_scene(sim_time_step):
    cat_path = './catBot.urdf'

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    # Make builder, plant, parser
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=sim_time_step)
    parser = Parser(plant)

    # Add cat
    parser.AddModelFromFile(cat_path)

    plant.Finalize()

    # Create plant context
    plant_context = plant.CreateDefaultContext()
    cat = plant.GetBodyByName('link_A')
    X_worldCat = RigidTransform(RollPitchYaw(np.array([0, 0, 0])), p=np.array([0, 0, 0.1]))
    plant.SetDefaultFreeBodyPose(cat, X_worldCat)

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, MeshcatVisualizerParams(role=Role.kPerception, 
                                                               prefix="visual")
    )

    diagram = builder.Build()
    return diagram, visualizer 

create_scene(0.001)



<RigidBody_[float] name='link_A' index=1 model_instance=2>


In [6]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(sim_time_step):
    diagram, visualizer = create_scene(sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    finish_time = 0.1 if test_mode else 2.0
    simulator.AdvanceTo(finish_time)
    visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(sim_time_step=0.0001)